In order to further improve our model, it is time to add even more data to our model! 

In [1]:
import pandas as pd
from itertools import tee
import warnings
warnings.filterwarnings('ignore')

In [2]:
avg_income = pd.read_excel("..\\data\\washington_state_income_by_zipcode.xlsx", usecols = ["Zip Code", "Location", "City", "Population", "Avg. Income/H/hold", "National Rank"])
avg_income.drop(index = [100, 201, 302, 403, 504], inplace = True)

In [3]:
avg_income.sample(5)

,Zip Code,Location,City,Population,Avg. Income/H/hold,National Rank
187,98601,"45.933373, -122.453150","Amboy, Washington",2327,45380,"#8,153"
534,98581,"46.369609, -123.071611","Ryderwood, Washington",422,24167,"#28,702"
44,99019,"47.642805, -117.079512","Liberty Lake, Washington",5370,60667,"#2,856"
186,98674,"45.951267, -122.665541","Woodland, Washington",9065,45452,"#8,119"
176,98371,"47.199448, -122.323541","Puyallup, Washington",20215,46268,"#7,602"


In [4]:
avg_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562 entries, 0 to 566
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Zip Code            562 non-null    object
 1   Location            562 non-null    object
 2   City                562 non-null    object
 3   Population          562 non-null    object
 4   Avg. Income/H/hold  562 non-null    object
 5   National Rank       562 non-null    object
dtypes: object(6)
memory usage: 30.7+ KB


In [5]:
avg_income["Zip Code"] = avg_income["Zip Code"].astype("int")

In [64]:
['Population', "Avg. Income/H/hold", "National Rank"]

['Population', 'Avg. Income/H/hold', 'National Rank']

In [66]:
avg_income['Population'] = avg_income['Population'].astype('int')

In [67]:
avg_income['Avg. Income/H/hold'] =  avg_income['Avg. Income/H/hold'].astype('int')

In [73]:
import re
avg_income['National Rank'] = avg_income['National Rank'].apply(lambda x: int(re.sub(r'[^\w\s]', '', x)))

In [74]:
avg_income

,Zip Code,Location,City,Population,Avg. Income/H/hold,National Rank
0,98039,"47.626035, -122.233707","Medina, Washington",3050,132665,46
1,98053,"47.655058, -122.025817","Redmond, Washington",31203,96028,335
2,98364,"47.855439, -122.595728","Port Gamble, Washington",102,94366,364
3,98040,"47.562356, -122.226514","Mercer Island, Washington",22036,91904,415
4,98029,"47.578444, -122.013924","Issaquah, Washington",28427,91146,430
...,...,...,...,...,...,...
562,98402,"47.251349, -122.443181","Tacoma, Washington",4935,15847,31304
563,99359,"46.510216, -118.194986","Starbuck, Washington",72,13393,31550
564,98174,"47.604569, -122.335359","Seattle, Washington",180,0,31889
565,98154,"47.606211, -122.333792","Seattle, Washington",1,0,31893


In [63]:
avg_income["Location"][0].replace("''" , "")

'47.626035, -122.233707'

In [6]:
homes = pd.read_csv("..\\data\\king_county_cleaned.csv", usecols= ['id', 'date', 'price', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'renov_yr_clean', 'view_ord',
       'condition_ord', 'grade_ord', 'age'])


In [48]:

x,y = avg_income.Location[0].split()[0][:9], avg_income.Location[0].split()[1]

x, y

('47.626035', '-122.233707')

In [7]:
homes.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,...,yr_renovated,lat,long,sqft_living15,sqft_lot15,renov_yr_clean,view_ord,condition_ord,grade_ord,age
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,1180,...,0.0,47.5112,-122.257,1340,5650,1955.0,0.0,2.0,4.0,60
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,2170,...,1991.0,47.7210,-122.319,1690,7639,1991.0,0.0,2.0,4.0,64
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,770,...,0.0,47.7379,-122.233,2720,8062,1933.0,0.0,2.0,3.0,82
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,1050,...,0.0,47.5208,-122.393,1360,5000,1965.0,0.0,4.0,4.0,50
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,1680,...,0.0,47.6168,-122.045,1800,7503,1987.0,0.0,2.0,5.0,28


In [8]:
lat_long = homes[['lat', 'long']]

In [9]:
#pairwise function implemented to iterate through two consecutive rows (pairs) in a data frame
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


In [10]:
#empty list - will be used to store calculated distances
list = [0]

# Loop through each row in the data frame using pairwise
for (i1, row1), (i2, row2) in pairwise(homes.iterrows()):
    #Assign latitude and longitude as origin/departure points
    LatOrigin = row1['lat']
    LongOrigin = row1['long']
    point = LatOrigin, LongOrigin
    list.append(point)
#Add column 'Distance' to data frame and assign to list values
homes['Location'] = list

In [11]:
homes['Location'][0] = 47.5112, -122.257

In [17]:
#homes["Location"] = homes["Location"].apply(lambda string: string.replace('(','').string(')',''))

In [13]:
homes

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,...,lat,long,sqft_living15,sqft_lot15,renov_yr_clean,view_ord,condition_ord,grade_ord,age,Location
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,1180,...,47.5112,-122.257,1340,5650,1955.0,0.0,2.0,4.0,60,"(47.5112, -122.257)"
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,2170,...,47.7210,-122.319,1690,7639,1991.0,0.0,2.0,4.0,64,"(47.5112, -122.257)"
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,770,...,47.7379,-122.233,2720,8062,1933.0,0.0,2.0,3.0,82,"(47.721, -122.319)"
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,1050,...,47.5208,-122.393,1360,5000,1965.0,0.0,4.0,4.0,50,"(47.7379, -122.233)"
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,1680,...,47.6168,-122.045,1800,7503,1987.0,0.0,2.0,5.0,28,"(47.5208, -122.393)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,0,1530,...,47.6993,-122.346,1530,1509,2009.0,0.0,2.0,5.0,6,"(47.5773, -122.409)"
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,0,2310,...,47.5107,-122.362,1830,7200,2014.0,0.0,2.0,5.0,1,"(47.6993, -122.346)"
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,0,1020,...,47.5944,-122.299,1020,2007,2009.0,0.0,2.0,4.0,6,"(47.5107, -122.362)"
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,0,1600,...,47.5345,-122.069,1410,1287,2004.0,0.0,2.0,5.0,11,"(47.5944, -122.299)"
